# Imagenet

Notebook created in PyTorch by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) for the UPC School (2020), and updated by [Pol Caselles](https://www.linkedin.com/in/pcaselles/) (2022) and [Gerard I. Gállego](https://www.linkedin.com/in/gerard-gallego/) (2022).

For this and many of the following labs we will use [Torchvision](https://pytorch.org/vision/stable/index.html), a library with computer vision datasets, pretrained models and useful methods made to work with PyTorch.

In [ ]:
import ast
import shutil
import urllib
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image

from typing import Tuple, List

from matplotlib.pyplot import imshow
torch.set_grad_enabled(True)
%matplotlib inline

To ensure reproducibility of the experiments, we can set the seed to a fixed number.

In [ ]:
seed = 123
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

In [ ]:
# we select to work on GPU

if not torch.cuda.is_available():
       raise RuntimeError("You should enable GPU runtime!!")
device = torch.device("cuda")

# whenever we send something to the selected device (X.to(device)) we already use
# either CPU or CUDA (GPU). Importantly...
# The .to() operation is in-place for nn.Module's, so network.to(device) suffices
# The .to() operation is NOT in.place for tensors, so we must assign the result
# to some tensor, like: X = X.to(device)

# Training on Imagenet

Since Imagenet is a really big dataset, we cannot train a model on it (it would take too much time in a single GPU). Anyway, we can take a look at how this training is done in PyTorch. We will go through a very similar version to the official code that was used to train the models from [Torchvision](https://pytorch.org/docs/stable/torchvision/index.html). The code of this notebook can also serve as a template for training your own models.

## Utils

First, we will define some useful functions. Make sure you understand what they do and how they work.

In [ ]:
def adjust_learning_rate(
        optimizer: torch.optim, 
        epoch: int, 
        original_lr: float
        ) -> None:
    
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = original_lr * (0.1 ** (epoch // 30))
    # For some models, different parameters are in different groups with different lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(
        output: torch.tensor, 
        target: torch.tensor, 
        topk: tuple =(1,)
        ) -> 'list[torch.tensor]':
    
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res  # res is a list of all the top k accuracies

def save_checkpoint(
        state: 'dict', 
        is_best: bool, 
        filename: str = 'checkpoint.pth.tar'
        ) -> None:
    
    torch.save(state, filename)
    
    # save an extra copy if it is the best model yet
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')  

## Model and optimizer

Here we will define the model. In our case, we will just take a predifined architecture from `torchvision.models`. You can take a look at the available models in: https://pytorch.org/vision/stable/models.html#classification

### Exercise 1

Set up the model and optimizer. Take a ResNet34 from `torchvision.models` with 100 classes (we will not train on the full ImageNet) and make sure it is on GPU. Note that you can set the number of classes with the parameter [`num_classes`](https://github.com/pytorch/vision/blob/0dceac025615a1c2df6ec1675d8f9d7757432a49/torchvision/models/resnet.py#L171). Then, declare an Adam optimizer with learning rate 0.001. You should also define the loss function for classification. The input of the loss will be the raw, unnormalized scores for each class.

In [ ]:
# TODO: take the ResNet34 model
model = ...

# TODO: declare Adam optimizer
optimizer = ...
# TODO: define the loss function
criterion = ...

try:
    assert isinstance(model, models.ResNet)
except AssertionError:
    raise Exception("Did you get an instance of a resnet34?")

try:
    assert model.fc.weight.shape[0] == 100
except AssertionError:
    raise Exception("Did you set the number of classes to 100?")

try:
    assert next(model.parameters()).is_cuda
except AssertionError:
    pass
    # raise Exception("Did you forget to move the model to GPU?")

print("Well done!")

## Data loading

Now we will prepare the dataset. A very important part of training on ImageNet is the data augmentation. The `torchvision.transforms` package has the transformations we need: https://pytorch.org/vision/stable/transforms.html 

More specifically, for training, we will:
* Do a RandomResizedCrop of size 224. This means, randomly resize the Image and then crop a square of size 224.
* Randomly flip the image horizontally.
* Converting our image to a torch tensor 
* Normalizing the values of the pixels with Imagenet mean and standard deviation for each channel.

And for validation we will:
* Resize our image to size 256 (this means, the smalles side will be 256)
* Cropping the image at the center with size 224
* Converting our image to a torch tensor 
* Normalizing the values of the pixels with Imagenet mean and standard deviation for each channel.

The ImageNet mean and std values are: `mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]`

### Exercise 2
Complete the code of the data augmentation

In [ ]:
# TODO: define the composed transformation for training
train_transforms = transforms.Compose([
    ...,
    ...,
    transforms.ToTensor(),
    transforms.Normalize(...),
])

# TODO: define the composed transformation for validation
val_transforms = transforms.Compose([
    ...,
    ...,
    transforms.ToTensor(),
    transforms.Normalize(...) ,
])

Now we can download the data and declare our dataloaders. We will use CIFAR100 to mimic ImageNet, since ImageNet has over 14 million images.

In [ ]:
train_dataset = datasets.CIFAR100(
    root='data', 
    train=True, 
    download=True, 
    transform=train_transforms,
)

val_dataset = datasets.CIFAR100(
    root='data', 
    train=False, 
    download=True, 
    transform=val_transforms,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True,
    num_workers=2, 
    pin_memory=True,
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=64, 
    shuffle=False, 
    num_workers=2,
)  # There is no need to shuffle data in validation


## Training

First we will define our train and validate functions. Once they are done, we can do the training loop.

### Exercise 3
Complete the necessary code for the train and validate functions.

In [ ]:
def train(
        train_loader: torch.utils.data.DataLoader, 
        model: torch.nn.Module, 
        criterion: torch.nn, 
        optimizer: torch.optim, 
        device: torch.device
        ) -> None:

    # TODO: switch to train mode
    ...

    for i, (images, target) in enumerate(train_loader):
        
        # Set network gradients to 0.
        # TODO:  reset gradients
        optimizer.

        # move images to gpu
        images = images.to(device)
        target = target.to(device)

        # Forward batch of images through the network
        # TODO: compute output
        output = ...

        # TODO: loss
        loss = ...

        # TODO: compute gradient and do optimization step
        loss.
        optimizer.
        
        # measure accuracy
        acc1, acc5 = accuracy(output, target, topk=(1, 5))

        if (i+1) % 50 == 0:
            print(f'TRAIN [{i+1}/{len(train_loader)}] Acc@1 {acc1.item():.3f} Acc@5 {acc5.item():.3f} Loss {loss.item():.3f}')


In [ ]:
def validate(
        val_loader: torch.utils.data.DataLoader, 
        model: torch.nn.Module, 
        criterion: torch.nn, 
        device: torch.device
        ) -> None:

    # TODO: switch to evaluate mode
    ...

    # We will save the values of the accuracies in this list to return the mean of the whole dataset at the end
    top1_scores = []  

    with torch.no_grad():  # We do not need to compute gradients
        for i, (images, target) in enumerate(val_loader):
            
            # move images to gpu
            images = images.to(device)
            target = target.to(device)

            # Forward batch of images through the network
            # TODO: compute output
            output = ...

            # TODO: loss
            loss = ...

            # measure accuracy
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            top1_scores.append(acc1.item()*len(target))

            if (i+1) % 50 == 0:
                print(f'VAL [{i+1}/{len(val_loader)}] Acc@1 {acc1.item():.3f} Acc@5 {acc5.item():.3f} Loss {loss.item():.3f}')
    
    return sum(top1_scores)/len(val_loader.dataset)

Finally, we are ready to train. 

### Exercise 4
Complete the training loop using the train and validate functions. You do not need to finish the training (even on CIFAR100 it can take hours).

In [ ]:
epochs = 1
best_acc1 = 0.
for epoch in range(epochs):
    adjust_learning_rate(optimizer, epoch, 0.001)

    # TODO: train for one epoch
    train(...)

    # TODO: evaluate on validation set
    acc1 = validate(...)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "resnet34",
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)


Good job! Now you know how ImageNet models are trained. Now you will learn how to use the models that are already pretrained in `torchvision.models`.

# Pretrained imagenet models

In [ ]:
# Disable gradient computation for this second half of the notebook
_ = torch.set_grad_enabled(False);

In Torchvision there are many models pretrained with Imagenet. You can see the lists [here](https://pytorch.org/vision/stable/models.html#table-of-all-available-classification-weights).
We will begin by seeing how to use these already trained networks to predict images taken from the internet. We will start by using a VGG16. 


### Exercise 5
Load a pretrained VGG16 model from PyTorch. Use the `weights` variable defined below. Remember to set validation mode calling `.eval()` and move the model to GPU.

In [ ]:
weights = models.VGG16_Weights.DEFAULT
# TODO : load a pretrained vgg16
model = ...

try:
    assert next(model.parameters()).is_cuda
except AssertionError:
    raise Exception("Did you forget to move the model to GPU?")

try:
    assert not model.training
except AssertionError:
    raise Exception("Did you forget set validation mode?")

print("Well done!")

We will also load the 1000 imagenet labels in a Python dictionary.

In [ ]:
urllib.request.urlretrieve("https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt", "labels.json")
with open("labels.json") as f:
    labels = ast.literal_eval(f.read()) # ast evaluates the string Python code
print(f"We have {len(labels)} labels.")
print(labels)

We can now download a sample image to test our network. When working with PyTorch, we usually use PIL or PILLOW (Python Image Library), which is the standard Pythonic way of working with images. `Image.open("\path\to\image.jpg")` returns an Image object, wich then can be converted to a Numpy or PyTorch tensor. We can look at the images using the `imshow` method from `matplotlib`.

In [ ]:
def show_image(pil_image):
    imshow(np.asarray(pil_image))


In [ ]:
urllib.request.urlretrieve("https://3c1703fe8d.site.internapcdn.net/newman/gfx/news/hires/2018/2-dog.jpg", "dog_image.jpg")
%ls

pil_image = Image.open("dog_image.jpg")
show_image(pil_image)


As you know, when the network was trained, ImageNet images went through some preprocessing transformations. Before feeding an image to the network, we should do the same transformations. 

As in validation, we will:
* Resize our image to size 256 (this means, the smallest side will be 256)
* Cropping the image at the center with size 224
* Converting our image to a torch tensor 
* Normalizing the values of the pixels with Imagenet mean and standard deviation for each channel. These are values are: `mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]`

We will use `transforms.Compose` to compose all these transformations in a single method.

### Exercise 6

Define the preprocessing transformations.

In [ ]:
# TODO : Define the preprocessing transformations defined above
preprocess = transforms.Compose([...])

torch_image = preprocess(pil_image)
try:
    assert isinstance(torch_image, torch.Tensor)
    assert list(torch_image.shape) == [3, 224, 224]
    print("Well done!")
except Exception:
    raise Exception("Did you do the 4 required transformations?")
    

We can take a look at the image that will go through the network.

In [ ]:
def show_torch_image(torch_image: torch.Tensor) -> None:
    img = (torch_image-torch_image.min()) / (torch_image.max() - torch_image.min())
    show_image(to_pil_image(img))

In [ ]:
show_torch_image(torch_image)

Now we can get the predictions of our network. Note that our model expects a batch dimension at the beginning, so we should add it with the method `.unsqueeze()`.

### Exercise 7
Complete the predict function. Remember to move the images to GPU, as well as adding the batch dimension.

In [ ]:
def predict_image(
        torch_image: torch.tensor, 
        model: torch.nn.Module, 
        topk: int = 3,
        ) -> List[Tuple[str, float]]:
    
    x = torch_image.to(device) # move image to GPU
    x = x.unsqueeze(0) # add batch dimension
    
    # TODO: predict raw outputs
    output = ...

    output = torch.softmax(output, dim=1)  #Compute the softmax to get probabilities
    probs, idxs = output.topk(topk)  # Get the top k predicitons
    return [(labels[i.item()], p.item()*100) for p, i in zip(probs[0], idxs[0])]

print(predict_image(torch_image, model))

### Exercise 8
Do the prediction with other images using different networks and compare the results.  You can use the function following function predict_from_url. The list of pretrained PyTorch networks can be found [here](https://pytorch.org/vision/stable/models.html#classification). When trying different models remember to move them to GPU and set evaluation mode.

In [ ]:
def predict_from_url(
        url: str, 
        model: torch.nn.Module,
        ) -> None:
    urllib.request.urlretrieve(url, "image.jpg")
    pil_image = Image.open("image.jpg")
    show_image(pil_image)
    torch_image = preprocess(pil_image)
    print(predict_image(torch_image, model))

predict_from_url("https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png", model)

In [ ]:
#new_model = models.vgg16(weights='DEFAULT').to(device).eval()
#new_model = models.ResNet(weights='DEFAULT').to(device).eval()

resnet18 = models.resnet18(weights='DEFAULT').to(device).eval()
#alexnet = models.alexnet(weights='DEFAULT')
#vgg16 = models.vgg16(weights='DEFAULT')
#squeezenet = models.squeezenet1_0(weights='DEFAULT')
#densenet = models.densenet161(weights='DEFAULT')
#inception = models.inception_v3(weights='DEFAULT')
#googlenet = models.googlenet(weights='DEFAULT')
#shufflenet = models.shufflenet_v2_x1_0(weights='DEFAULT')
#mobilenet_v2 = models.mobilenet_v2(weights='DEFAULT')
#mobilenet_v3_large = models.mobilenet_v3_large(weights='DEFAULT')
#mobilenet_v3_small = models.mobilenet_v3_small(weights='DEFAULT')
#resnext50_32x4d = models.resnext50_32x4d(weights='DEFAULT')
#wide_resnet50_2 = models.wide_resnet50_2(weights='DEFAULT')
#mnasnet = models.mnasnet1_0(weights='DEFAULT')

predict_from_url("https://icatcare.org/app/uploads/2018/07/Thinking-of-getting-a-cat.png", resnet18)